In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=58f045aa3a7dba0f16e0f53f5e0cee0a784dbebe3951db91659d3d703e01cd01
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.7MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F15 '''
def F15(X):
    f = bn.F15()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.607021964353885 ,random_state=0 , l1_ratio=0.9595959595959597, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_15_1000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_15_200Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [0]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,51):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 50, [5] * 50 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 50000)

n_obs , dim =  30, 50
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,150])
for i in range(G[1].shape[0]):
        min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
        min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
        min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
        X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=970438, Mon Apr 13 20:10:13 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 6.061410406482706e+03 1.0e+00 2.37e+00  2e+00  2e+00 0:00.0
    2     30 4.870033641308650e+03 1.0e+00 2.31e+00  2e+00  2e+00 0:00.0
    3     45 4.925463945742455e+03 1.1e+00 2.29e+00  2e+00  2e+00 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [7839.98263843] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [6203.7086874] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 -9.072932171947559e+03 1.4e+00 2.74e-01  3e-01  3e-01 0:01.7
  200   3000 -9.326885469802772e+03 1.4e+00 1.98e-02  2e-02  2e-02 0:03.3
  300   4500 -9.328513155980061e+03 1.5e+00 1.12e-03  1e-03  1e-03 0:05.0
  400   6000 -9.328517981600995e+03 1.5e+00 7.96e-05  7e-05  7e-05 0:06.6
  500   7500 -9.328518011188327e+03 1.6e+00 4.18e-06  3e-06  4e-06 0:08.3
  600   9000 -9.328518011244414e+03 1.6e+00 2.64e-07  2e-07  2e-07 0:09.9
  700  10500 -9.328518011244621e+03 1.7e+00 2.94e-08  2e-08  3e-08 0:11.6
  800  12000 -9.328518011244625e+03 1.9e+00 1.67e-08  1e-08  1e-08 0:13.2
  807  12105 -9.328518011244621e+03 1.9e+00 1.66e-08  1e-08  1e-08 0:13.3
termination on tolfun=1e-11 (Mon Apr 13 20:10:29 2020)
final/bestever f-value = -9.328518e+03 -9.328518e+03
incumbent solution: [ 0.48973292 -0.69596963  0.88243148  1.72170728 -0.60754259 -1.64740442
 -0.69831173 -1.56428464 ...]
std deviations: [1.34963425e-08 1.26022926e-08 1.26535067e-08 1.30416111e-08
 1.30285283e-08 1.30713642

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [5999.55830684] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=5999.558307, sigma=2.35e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=5999.558307, sigma=2.35e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')

  100   1500 -5.054552076910551e+03 1.5e+00 3.95e-01  4e-01  4e-01 0:02.0
  200   3000 -5.789733586239525e+03 1.7e+00 9.27e-02  8e-02  1e-01 0:04.0
  300   4500 -5.894171567077807e+03 3.0e+00 9.57e-02  7e-02  2e-01 0:06.0
  400   6000 -5.961271349581502e+03 3.3e+00 4.03e-02  3e-02  7e-02 0:07.9
  500   7500 -5.971484709238053e+03 3.8e+00 1.50e-02  1e-02  2e-02 0:09.9
  600   9000 -5.972745603537018e+03 4.3e+00 4.95e-03  3e-03  7e-03 0:11.9
  700  10500 -5.972821753879176e+03 4.5e+00 7.08e-04  4e-04  1e-03 0:13.9
  800  12000 -5.972823306542994e+03 4.7e+00 1.42e-04  8e-05  2e-04 0:15.9
  900  13500 -5.972823343320542e+03 5.0e+00 2.23e-05  1e-05  4e-05 0:17.9
 1000  15000 -5.972823344266506e+03 5.2e+00 3.37e-06  2e-06  5e-06 0:19.9
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -5.972823344288470e+03 5.4e+00 5.08e-07  2e-07  8e-07 0:21.9
 1200  18000 -5.972823344288809e+03 5.8e+00 6.14e-08  3e-08  1e-07 0:23.9
 1300  19500 -5.972823344288818e+03 6.3e

...

  100   1500 -4.949411921118761e+03 1.5e+00 5.02e-01  5e-01  5e-01 0:02.0
  200   3000 -5.629108291565879e+03 1.7e+00 1.13e-01  1e-01  1e-01 0:04.0
  300   4500 -5.850676061461032e+03 3.4e+00 1.21e-01  9e-02  2e-01 0:06.0
  400   6000 -5.927990238236453e+03 3.8e+00 2.87e-02  2e-02  5e-02 0:08.0
  500   7500 -5.931791908202693e+03 3.8e+00 8.74e-03  6e-03  1e-02 0:10.0
  600   9000 -5.933053044127753e+03 5.3e+00 8.32e-03  5e-03  2e-02 0:11.9
  700  10500 -5.933521128470880e+03 7.1e+00 3.44e-03  2e-03  1e-02 0:13.9
  800  12000 -5.933573823908418e+03 7.9e+00 1.41e-03  7e-04  4e-03 0:15.9
  900  13500 -5.933585093229196e+03 8.3e+00 5.47e-04  3e-04  2e-03 0:17.9
 1000  15000 -5.933587191245088e+03 1.3e+01 2.97e-04  1e-04  2e-03 0:19.9
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -5.933587442362095e+03 2.0e+01 8.60e-05  4e-05  7e-04 0:21.9
 1200  18000 -5.933587453872344e+03 2.3e+01 8.19e-06  3e-06  7e-05 0:23.9
 1300  19500 -5.933587453961743e+03 2.5e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [6129.69088088] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [4389.3570798] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 -9.175982929110760e+03 1.4e+00 2.04e-01  2e-01  2e-01 0:01.9
  200   3000 -9.328235080803974e+03 1.4e+00 7.95e-03  7e-03  8e-03 0:03.9
  300   4500 -9.328516914473868e+03 1.5e+00 4.76e-04  4e-04  4e-04 0:05.8
  400   6000 -9.328518008802406e+03 1.6e+00 2.65e-05  2e-05  2e-05 0:07.8
  500   7500 -9.328518011238455e+03 1.6e+00 1.38e-06  1e-06  1e-06 0:09.7
  600   9000 -9.328518011244596e+03 1.6e+00 9.02e-08  7e-08  8e-08 0:11.7
  700  10500 -9.328518011244625e+03 1.7e+00 1.92e-08  1e-08  2e-08 0:13.6
  764  11460 -9.328518011244621e+03 1.9e+00 1.38e-08  1e-08  1e-08 0:14.9
termination on tolfun=1e-11 (Mon Apr 13 20:37:46 2020)
final/bestever f-value = -9.328518e+03 -9.328518e+03
incumbent solution: [ 0.48973299 -0.69596968  0.88243142  1.72170725 -0.60754256 -1.64740433
 -0.6983118  -1.56428466 ...]
std deviations: [1.09782608e-08 1.02781217e-08 1.04113516e-08 1.08488696e-08
 1.11686012e-08 1.06602186e-08 1.07590945e-08 1.01271235e-08 ...]
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [5999.55830684] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=5999.558307, sigma=2.36e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=5999.558307, sigma=2.36e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')

  100   1500 -3.625493588560434e+03 1.5e+00 5.50e-01  5e-01  6e-01 0:02.0
  200   3000 -4.641434905670154e+03 1.7e+00 8.60e-02  7e-02  1e-01 0:04.0
  300   4500 -4.671209058438773e+03 2.3e+00 2.33e-02  2e-02  3e-02 0:05.9
  400   6000 -4.675905213866952e+03 3.8e+00 1.57e-02  1e-02  2e-02 0:07.9
  500   7500 -4.680765005925778e+03 6.9e+00 1.81e-02  1e-02  5e-02 0:09.9
  600   9000 -4.681920749733927e+03 9.0e+00 5.25e-03  3e-03  1e-02 0:11.9
  700  10500 -4.681989125355740e+03 9.7e+00 7.77e-04  4e-04  2e-03 0:13.8
  800  12000 -4.681991246792071e+03 1.0e+01 2.65e-04  1e-04  7e-04 0:15.8
  900  13500 -4.681991409199441e+03 1.1e+01 4.04e-05  2e-05  1e-04 0:17.8
 1000  15000 -4.681991412008816e+03 1.1e+01 4.36e-06  2e-06  1e-05 0:19.8
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -4.681991412041758e+03 1.2e+01 4.64e-07  2e-07  1e-06 0:21.7
 1200  18000 -4.681991412042022e+03 1.2e+01 5.63e-08  2e-08  1e-07 0:23.7
 1278  19170 -4.681991412042024e+03 1.3e

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F15_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F15(X_Values.iloc[i,:50])
    SVM_Fun [i] = F15(X_Values.iloc[i,50:100])
    ELN_Fun [i] = F15(X_Values.iloc[i,100:150])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([930.74894671]), count=array([1])) 2.306167922490213e-05
SVM
ModeResult(mode=array([2545.05535624]), count=array([1])) 3662.557311536977
ELN
ModeResult(mode=array([3817.67730871]), count=array([1])) 678.6648972685346
